# Rank and Trade

### MGMT 767, Quantitative Investments Lab
### Kerry Back & Kevin Crotty, Rice University

### Outline

1. Build feature dataset with today's features
2. Load and apply model to predict
3. Trade to 140/40 portfolio with 100% in SPY
   (a) Close unwanted positions
   (b) Rebalance SPY
   (c) Open/rebalance long positions
   (d) Open/rebalance short positions
4. Save account equity and positions

In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from joblib import load 
import yfinance as yf 
from datetime import datetime
import os.path

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, GetAssetsRequest, AssetClass
from alpaca.trading.enums import OrderSide, TimeInForce

### Build Feature Dataset

- Don't need much history.  Start here in 2022.
- And don't need weekly returns (after computing momentum).

In [2]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'
driver = 'SQL+Server'
string = f"mssql+pyodbc://{username}:{password}@{server}/{database}" 
try: 
    conn = create_engine(string + "?driver='SQL+Server'").connect()
except:
    try:
        conn = create_engine(string + "?driver='ODBC+Driver+18+for+SQL+Server'").connect()
    except:
        import pymssql
        string = f"mssql+pymssql://{username}:{password}@{server}/{database}"   
        conn = create_engine(string).connect() 

In [3]:
sep_weekly = pd.read_sql(
    """ 
    select date, ticker, closeadj, closeunadj, volume, lastupdated from sep_weekly 
    where date >= '2022-01-01'
    order by ticker, date, lastupdated    
    """,
    conn,
)
sep_weekly = sep_weekly.groupby(["ticker", "date"]).last()
sep_weekly = sep_weekly.drop(columns=["lastupdated"])

ret = sep_weekly.groupby("ticker", group_keys=False).closeadj.pct_change()
ret.name = "ret"

price = sep_weekly.closeunadj
price.name = "price"

volume = sep_weekly.volume 
volume.name = "volume"

In [4]:
sep_weekly.reset_index().date.max()

datetime.date(2024, 3, 29)

In [5]:
ret_annual = sep_weekly.groupby("ticker", group_keys=False).closeadj.pct_change(52)
ret_monthly = sep_weekly.groupby("ticker", group_keys=False).closeadj.pct_change(4)
mom = (1 + ret_annual) / (1 + ret_monthly) - 1
mom.name = "mom"

In [6]:
weekly = pd.read_sql(
    """ 
    select date, ticker, pb, marketcap, lastupdated from weekly 
    where date>='2022-01-01'
    order by ticker, date, lastupdated    
    """,
    conn,
)
weekly = weekly.groupby(["ticker", "date"]).last()
weekly = weekly.drop(columns=["lastupdated"])

pb = weekly.pb
pb.name = "pb" 
marketcap = weekly.marketcap 
marketcap.name = "marketcap"

In [7]:
weekly.reset_index().date.max()

datetime.date(2024, 3, 29)

In [8]:
sf1 = pd.read_sql(
    """ 
    select datekey as date, ticker, assets, netinc, equity, lastupdated from sf1
    where datekey>='2022-01-01' and dimension='ARY' and assets>0 and equity>0
    order by ticker, datekey, lastupdated    
    """,
    conn,
)
sf1 = sf1.groupby(["ticker", "date"]).last()
sf1 = sf1.drop(columns=["lastupdated"])

# change dates to Fridays
from datetime import timedelta 
sf1 = sf1.reset_index()
sf1.date =sf1.date.map(
    lambda x: x + timedelta(4 - x.weekday())
)
sf1 = sf1.set_index(["ticker", "date"])
sf1 = sf1[~sf1.index.duplicated()]

assets = sf1.assets
assets.name = "assets" 
netinc = sf1.netinc 
netinc.name = "netinc" 
equity = sf1.equity
equity.name = "equity"

equity = equity.groupby("ticker", group_keys=False).shift() 
roe = netinc / equity 
roe.name = "roe"

assetgr = assets.groupby("ticker", group_keys=False).pct_change()
assetgr.name = "assetgr"

In [9]:
sf1=sf1.reset_index()
sf1[sf1.date==sf1.date.max()]

,ticker,date,assets,netinc,equity
65,ABOS,2024-03-29,310125000.0,-52371000.0,266973000.0
159,ACRV,2024-03-29,138265000.0,-60388000.0,121195000.0
296,AGAE,2024-03-29,112016687.0,-3595361.0,82749924.0
300,AGBA,2024-03-29,66588967.0,-49206019.0,8102771.0
533,ALT,2024-03-29,210640000.0,-88447000.0,194099000.0
...,...,...,...,...,...
11720,WYY,2024-03-29,51300499.0,-4046473.0,14677429.0
11849,ZEO,2024-03-29,47987187.0,6230438.0,32332888.0
11901,ZUO,2024-03-29,823836000.0,-68193000.0,133687000.0
11902,ZURA,2024-03-29,100843000.0,-60360000.0,59330000.0


In [10]:
sf1.reset_index().date.max()

datetime.date(2024, 3, 29)

In [11]:
df = pd.concat(
    (
        mom, 
        volume,
        price, 
        pb, 
        marketcap, 
        roe, 
        assetgr
        ), 
        axis=1
    )
df["roe"] = df.groupby("ticker", group_keys=False).roe.ffill()
df["assetgr"] = df.groupby("ticker", group_keys=False).assetgr.ffill()

df = df.reset_index()
df.date = df.date.astype(str)
df = df[df.date==df.date.max()]
df = df[df.price >= 5]
df = df.dropna()

features = [
    "mom",
    "volume",
    "pb",
    "marketcap",
    "roe",
    "assetgr" 
]

In [12]:
industries = pd.read_sql(
    """ 
    select ticker, famaindustry as industry from tickers   
    """,
    conn,
)
industries["industry"] = industries.industry.fillna("Almost Nothing")
df = df.merge(industries, on="ticker", how="left")
df = df.dropna()

In [13]:
for x in features:
    df[f"{x}_industry"] = df.groupby(
        ["industry"], 
        group_keys=False
    )[x].apply(
        lambda x: x - x.median()
    )

features += [f"{x}_industry" for x in features]

In [14]:
for f in features:
    df[f] = df[f].rank(pct=True)

### Load Model and Predict

In [15]:
model = load("mymodel.joblib")
df["predict"] = model.predict(df[features])

### Best and worst stocks

- Best stocks must be tradable 
- Worst stocks must be tradable and shortable

In [16]:
with open("keys.txt", "r") as f:
    keys = f.readlines()

key, secret_key = [x.strip() for x in keys]
trading_client = TradingClient(key, secret_key, paper=True)

search_params = GetAssetsRequest(asset_class=AssetClass.US_EQUITY)
assets = trading_client.get_all_assets(search_params)
tradable = [x.symbol for x in assets if x.tradable]
shortable = [x.symbol for x in assets if x.shortable]

In [17]:
numstocks = 50

df = df.sort_values(by="predict", ascending=False)
best = df[["ticker", "predict"]].copy().reset_index(drop=True)
best = best[best.ticker.isin(tradable)].iloc[:numstocks]

df = df.sort_values(by="predict", ascending=True)
worst = df[["ticker", "predict"]].copy().reset_index(drop=True)
worst = worst[worst.ticker.isin(shortable)].iloc[:numstocks]

In [18]:
best

,ticker,predict
0,VRT,51.965860
1,CVNA,51.903699
2,SMCI,51.803189
3,CLX,51.377378
4,CTAS,51.375721
5,TSCO,51.370101
6,CDNS,51.359203
7,COST,51.359203
8,BR,51.353720
9,ARES,51.352391


In [19]:
worst

,ticker,predict
2,EGIO,45.090538
8,ONCT,45.469028
11,EP,45.578664
16,VRM,45.786216
18,LPCN,45.867946
21,QTTB,46.054224
22,NVCT,46.115831
25,AIRG,46.219827
27,SCOR,46.408658
28,ELVN,46.414920


### Close unwanted positions

In [30]:
positions = trading_client.get_all_positions()
positions = {x.symbol: float(x.qty) for x in positions}
positions_to_close = [
    symbol for symbol in positions 
    if (symbol not in best.ticker.to_list())
    and (symbol not in worst.ticker.to_list())
    and (symbol != "SPY")
]

for symbol in positions_to_close:
    qty = positions[symbol]
    order=MarketOrderRequest(
        symbol=symbol,
        qty=abs(qty),
        side=OrderSide.BUY if qty<0 else OrderSide.SELL,
        time_in_force=TimeInForce.DAY
    )
    _ = trading_client.submit_order(order)

### Rebalance SPY

In [31]:
price = yf.download("SPY", start="2024-01-01", progress=False)["Close"].iloc[-1].item()

account = trading_client.get_account()
equity = float(account.equity)
qty = int(equity / price)
qty -= positions["SPY"] if "SPY" in positions else 0

if qty != 0:
    order = MarketOrderRequest(
        symbol="SPY",
        qty=abs(qty),
        side=OrderSide.BUY if qty>0 else OrderSide.SELL,
        time_in_force=TimeInForce.DAY
    )
    _ = trading_client.submit_order(order)

Exception in thread Thread-4 (_run_via_pool):
Traceback (most recent call last):
  File "c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\site-packages\multitasking\__init__.py", line 104, in _run_via_pool
    return callee(*args, **kwargs)
  File "c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\multi.py", line 188, in _download_one_threaded
    data = _download_one(ticker, start, end, auto_adjust, back_adjust,
  File "c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\multi.py", line 202, in _download_one
    return Ticker(ticker).history(period=period, interval=interval,
  File "c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\site-

: 

### Trade best stocks

In [ ]:
symbols = best.ticker.to_list()
prices = yf.download(symbols, start=2024)["Close"].iloc[-1]
symbols = [s for s in symbols if not np.isnan(prices[s])]
dollars = 0.4 * equity / numstocks
for symbol in symbols:
    price = prices[symbol]
    qty = int(dollars / price)
    qty -= positions[symbol] if symbol in positions else 0
    if qty != 0:
        try:
            order = MarketOrderRequest(
                symbol=symbol,
                qty=abs(qty),
                side=OrderSide.BUY if qty>0 else OrderSide.SELL,
                time_in_force=TimeInForce.DAY
            )
            _ = trading_client.submit_order(order)
        except Exception as error:
            print("An error occurred:", error)

[*********************100%%**********************]  50 of 50 completed


### Trade worst stocks

In [ ]:
symbols = worst.ticker.to_list()
prices = yf.download(symbols, start=2024)["Close"].iloc[-1]
symbols = [s for s in symbols if not np.isnan(prices[s])]
for symbol in symbols:
    price = prices[symbol]
    qty = - int(dollars / price)
    qty -= positions[symbol] if symbol in positions else 0
    if qty != 0:
        try:
            order = MarketOrderRequest(
                symbol=symbol,
                qty=abs(qty),
                side=OrderSide.BUY if qty>0 else OrderSide.SELL,
                time_in_force=TimeInForce.DAY
            )
            _ = trading_client.submit_order(order)
        except Exception as error:
            print("An error occurred:", error)

[*********************100%%**********************]  50 of 50 completed


### Save data

In [ ]:
today = datetime.strftime(datetime.today(), "%Y-%m-%d")
account = trading_client.get_account()
equity = float(account.equity)
if os.path.isfile("equity.csv"):
    d = pd.read_csv("equity.csv", index_col="date")
    d.loc[today] = equity 
else: 
    d = pd.Series({today: equity})
    d.name = "equity" 
    d.index.name = "date"
d.to_csv("equity.csv")

In [ ]:
positions = trading_client.get_all_positions()
d = pd.DataFrame([x.qty for x in positions], index=[x.symbol for x in positions], columns=["shares"])
d["date"] = today
d.index.name = "symbol"
d = d.reset_index()
if os.path.isfile("positions.csv"):
    d0 = pd.read_csv("positions.csv")
    d = pd.concat((d0, d))
d.to_csv("positions.csv", index=False)